In [1]:
import json
import warnings
from utils import generate_lattice
import geopandas as gpd
import numpy as np
import pandas as pd
from shapely.geometry import Point, box

# Suppress deprecation warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
with open("../data/standard_variables.json", "r") as f:
    standard_variables = json.load(f)

standard_variables

{'map_limits': {'north': 64.188968,
  'south': 64.033992,
  'east': -21.640345,
  'west': -22.083955},
 'grid_size': {'unit': 'meters', 'value': 100},
 'standard_variables': {'cutoff_radius': {'unit': 'meters', 'value': 400}}}

In [3]:


map_limits = standard_variables["map_limits"]
grid_size = standard_variables["grid_size"]["value"]

lattice_points = generate_lattice(map_limits, grid_size)

print(f"Generated {len(lattice_points)} lattice points")
print(lattice_points[:5])

Generated 195624 lattice points
[(-22.083955, 64.033992), (-22.083056684715878, 64.033992), (-22.08215836943176, 64.033992), (-22.08126005414764, 64.033992), (-22.08036173886352, 64.033992)]


In [4]:
def estimate_parking_for_lattice_optimized(gdf, points, radius):
    """
    Efficiently estimate parking capacity for a lattice of points.

    Parameters:
    - gdf (GeoDataFrame): GeoDataFrame containing parking lot data with an 'estimated_capacity' column.
    - points (list of tuples): List of (longitude, latitude) tuples representing locations to search near.
    - radius (float): The search radius in meters.

    Returns:
    - results (list of dict): List containing total capacity and nearby parking lots for each point.
    """
    # Reproject GeoDataFrame to metric CRS only once
    gdf_metric = gdf.to_crs(epsg=3857)
    spatial_index = gdf_metric.sindex  # Build spatial index once

    # Reproject all points to metric CRS at once
    points_metric = gpd.GeoSeries(
        [Point(pt) for pt in points], crs=gdf.crs
    ).to_crs(epsg=3857)

    results = []
    for idx, point in enumerate(points_metric):
        # Create a buffer around the point
        search_area = point.buffer(radius)

        # Use the spatial index to find potential matches
        possible_matches_index = list(
            spatial_index.intersection(search_area.bounds))
        possible_matches = gdf_metric.iloc[possible_matches_index]

        # Filter with exact intersection
        nearby_parking = possible_matches[possible_matches.intersects(
            search_area)]

        # Calculate the total estimated capacity
        total_capacity = nearby_parking['estimated_capacity'].sum()

        results.append({
            "point_index": idx,
            "location": points[idx],
            "total_capacity": total_capacity,
            "nearby_parking": nearby_parking
        })

    return results

In [5]:
gdf_parkings = gpd.read_file("../data/parking_lots.geojson")

# Check if the CRS is correct (should be WGS84 for compatibility with the function)
if gdf_parkings.crs != "EPSG:4326":
    gdf_parkings = gdf_parkings.to_crs(epsg=4326)

display(gdf_parkings)

,element_type,osmid,access,amenity,capacity,fee,parking,surface,operator,covered,capacity:disabled,calculated_area,estimated_capacity,geometry
0,way,4347334,None,parking,None,None,street_side,None,None,None,None,597.881119,3.477211,"MULTIPOLYGON (((-21.93892 64.15974, -21.93884 ..."
1,way,4370534,yes,parking,None,no,surface,None,None,None,None,1990.194635,11.574755,"MULTIPOLYGON (((-21.92879 64.14502, -21.92844 ..."
2,way,4383232,None,parking,None,None,None,None,None,None,None,2264.271066,13.168754,"MULTIPOLYGON (((-21.97168 64.14855, -21.97167 ..."
3,way,4403322,yes,parking,None,no,surface,asphalt,None,None,yes,13498.502969,78.505823,"MULTIPOLYGON (((-21.92808 64.12218, -21.92814 ..."
4,way,4403323,None,parking,None,None,None,None,None,None,None,4739.967403,27.567134,"MULTIPOLYGON (((-21.92845 64.12237, -21.92956 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4414,relation,11762330,yes,parking,None,no,street_side,None,None,None,1,610.990884,3.553456,"MULTIPOLYGON (((-21.93391 64.14635, -21.93420 ..."
4415,relation,12668440,yes,parking,None,no,surface,asphalt,None,None,None,27053.659708,157.341138,"MULTIPOLYGON (((-21.90188 64.13776, -21.90088 ..."
4416,relation,12859326,yes,parking,None,no,multi-storey,paved,None,None,None,50861.404633,295.804389,"MULTIPOLYGON (((-21.84834 64.14296, -21.84833 ..."
4417,relation,16810737,yes,parking,None,no,surface,asphalt,None,None,None,2145.629491,12.478747,"MULTIPOLYGON (((-21.66358 64.16824, -21.66344 ..."


In [6]:
results = estimate_parking_for_lattice_optimized(
    gdf_parkings, lattice_points, standard_variables['standard_variables']['cutoff_radius']['value'])

# Display results
for result in results[:5]:
    print(f"Point {result['point_index']} (Location: {result['location']}):")
    print(f"  Total parking capacity: {result['total_capacity']}")

Point 0 (Location: (-22.083955, 64.033992)):
  Total parking capacity: 0.0
Point 1 (Location: (-22.083056684715878, 64.033992)):
  Total parking capacity: 0.0
Point 2 (Location: (-22.08215836943176, 64.033992)):
  Total parking capacity: 0.0
Point 3 (Location: (-22.08126005414764, 64.033992)):
  Total parking capacity: 0.0
Point 4 (Location: (-22.08036173886352, 64.033992)):
  Total parking capacity: 0.0


In [7]:
# Prepare the results for saving
results_list = []
for result in results:
    results_list.append({
        "point_index": result["point_index"],
        "latitude": result["location"][1],
        "longitude": result["location"][0],
        "total_capacity": result["total_capacity"]
    })

# Convert to DataFrame
results_df = pd.DataFrame(results_list)

# Save as CSV
results_df.to_csv("../data/lattice_parking_capacity.csv", index=False)

print("Results have been saved as 'lattice_parking_capacity.csv'.")

Results have been saved as 'lattice_parking_capacity.csv'.


In [8]:
gpd_district_geom = gpd.read_file('../data/pop_district_geom_fixed.geojson')
display(gpd_district_geom)

,smsv,smsv_label,smsv_label_en,Medaltal,geometry
0,0101,Reykjavík: Vesturbær norður - 0101,Reykjavik: Vesturbaer north - 0101,1379,"MULTIPOLYGON (((-21.95516 64.14637, -21.95490 ..."
1,0102,Reykjavík: Vesturbær norður - 0102,Reykjavik: Vesturbaer north - 0102,1243,"MULTIPOLYGON (((-21.94822 64.14279, -21.94802 ..."
2,0103,Reykjavík: Vesturbær norður - 0103,Reykjavik: Vesturbaer north - 0103,1267,"MULTIPOLYGON (((-21.94597 64.14783, -21.94475 ..."
3,0104,Reykjavík: Vesturbær norður - 0104,Reykjavik: Vesturbaer north - 0104,1071,"MULTIPOLYGON (((-21.94609 64.15015, -21.94430 ..."
4,3101,Suðurnes án Reykjanesbæjar - 3101,Southwest excl. Reykjanesbaer - 3101,2093,"MULTIPOLYGON (((-22.42807 63.94181, -22.42805 ..."
...,...,...,...,...,...
200,3402,Vestfirðir - 3402,Westfjords - 3402,1075,"MULTIPOLYGON (((-22.65863 66.09024, -22.65748 ..."
201,0705,Reykjavík: Háaleiti og Bústaðahverfi - 0705,Reykjavik: Haaleiti and Bustadahverfi - 0705,1892,"MULTIPOLYGON (((-21.87431 64.11726, -21.87431 ..."
202,3303,Vesturland án Akraness - 3303,West excl. Akraness - 3303,1437,"MULTIPOLYGON (((-20.87857 64.70605, -20.87973 ..."
203,0603,Reykjavík: Laugardalur austur - 0603,Reykjavik: Laugardalur east - 0603,924,"MULTIPOLYGON (((-21.87529 64.14680, -21.87528 ..."


In [9]:
points_gdf = gpd.GeoDataFrame(
    geometry=[Point(x, y) for x, y in lattice_points],
    crs="EPSG:4326"
)

# Ensure the district GeoDataFrame has the same CRS
if gpd_district_geom.crs != points_gdf.crs:
    gpd_district_geom = gpd_district_geom.to_crs(points_gdf.crs)

# Perform the spatial join to find which district each point belongs to
district_gdf = gpd.sjoin(points_gdf, gpd_district_geom,
                         how="left", predicate="within")

# Retrieve the `smsv` and other relevant information
result = district_gdf[["geometry", "smsv",
                       "smsv_label", "smsv_label_en", "Medaltal"]]

print(result)

                          geometry  smsv                      smsv_label  \
0       POINT (-22.08395 64.03399)  1501  Hafnarfjörður: Suðurbær - 1501   
1       POINT (-22.08306 64.03399)  1501  Hafnarfjörður: Suðurbær - 1501   
2       POINT (-22.08216 64.03399)  1501  Hafnarfjörður: Suðurbær - 1501   
3       POINT (-22.08126 64.03399)  1501  Hafnarfjörður: Suðurbær - 1501   
4       POINT (-22.08036 64.03399)  1501  Hafnarfjörður: Suðurbær - 1501   
...                            ...   ...                             ...   
195619  POINT (-21.64468 64.18892)  2401      Mosfellsbær og Kjós - 2401   
195620  POINT (-21.64378 64.18892)  2401      Mosfellsbær og Kjós - 2401   
195621  POINT (-21.64288 64.18892)  2401      Mosfellsbær og Kjós - 2401   
195622  POINT (-21.64198 64.18892)  2401      Mosfellsbær og Kjós - 2401   
195623  POINT (-21.64109 64.18892)  2401      Mosfellsbær og Kjós - 2401   

                           smsv_label_en Medaltal  
0       Hafnarfjoerdur: Sudurbaer -

In [10]:
district_gdf.to_file("../data/district_lattice.geojson", driver='GeoJSON')

In [11]:
buildings_with_population = gpd.read_file(
    "../data/buildings_with_population.geojson")
display(buildings_with_population)

,smsv,calculated_area,Medaltal,total_area,growth,population_2025,population_2029,population_2030,geometry
0,0405,3883.959942,2619.0,3.717841e+05,1.000000,27.360212,27.360212,27.360212,"MULTIPOLYGON (((-21.91400 64.14308, -21.91387 ..."
1,2105,4752.141405,1154.0,3.757481e+05,1.338843,14.594807,19.540155,26.161199,"MULTIPOLYGON (((-21.88009 64.10280, -21.87926 ..."
2,2105,56989.880228,1154.0,3.757481e+05,1.338843,175.027683,234.334584,313.737211,"MULTIPOLYGON (((-21.88060 64.10358, -21.87837 ..."
3,0405,3505.833698,2619.0,3.717841e+05,1.000000,24.696535,24.696535,24.696535,"MULTIPOLYGON (((-21.91381 64.14119, -21.91343 ..."
4,0401,2448.573105,2258.0,4.778074e+05,1.000000,11.571352,11.571352,11.571352,"MULTIPOLYGON (((-21.89905 64.11787, -21.89931 ..."
...,...,...,...,...,...,...,...,...,...
47165,0501,9431.840269,2628.0,5.635955e+05,1.202532,43.979903,52.887225,63.598561,"MULTIPOLYGON (((-21.90116 64.14565, -21.90108 ..."
47166,1203,3382.092624,1693.0,2.793860e+05,1.012821,20.494525,20.757275,21.023394,"MULTIPOLYGON (((-21.79414 64.15055, -21.79434 ..."
47167,1101,9086.683094,1467.0,1.103674e+06,1.000000,12.077992,12.077992,12.077992,"MULTIPOLYGON (((-21.82900 64.12973, -21.82904 ..."
47168,1101,15493.000624,1467.0,1.103674e+06,1.000000,20.593251,20.593251,20.593251,"MULTIPOLYGON (((-21.82823 64.12915, -21.82902 ..."


In [12]:
def estimate_population_for_lattice_optimized(gdf, points, radius):
    """
    Efficiently estimate population in proximity to a lattice of points.

    Parameters:
    - gdf (GeoDataFrame): GeoDataFrame containing population data with 'population_2025', 'population_2029', 'population_2030' columns.
    - points (list of tuples): List of (longitude, latitude) tuples representing locations to search near.
    - radius (float): The search radius in meters.

    Returns:
    - results (list of dict): List containing total population and nearby buildings for each point.
    """
    # Reproject GeoDataFrame to metric CRS only once
    gdf_metric = gdf.to_crs(epsg=3857)
    spatial_index = gdf_metric.sindex  # Build spatial index once

    # Reproject all points to metric CRS at once
    points_metric = gpd.GeoSeries(
        [Point(pt) for pt in points], crs=gdf.crs
    ).to_crs(epsg=3857)

    results = []
    for idx, point in enumerate(points_metric):
        # Create a buffer around the point
        search_area = point.buffer(radius)

        # Use the spatial index to find potential matches
        possible_matches_index = list(
            spatial_index.intersection(search_area.bounds))
        possible_matches = gdf_metric.iloc[possible_matches_index]

        # Filter with exact intersection
        nearby_buildings = possible_matches[possible_matches.intersects(
            search_area)]

        # Calculate the total population for each year
        total_population_2025 = nearby_buildings['population_2025'].sum()
        total_population_2029 = nearby_buildings['population_2029'].sum()
        total_population_2030 = nearby_buildings['population_2030'].sum()

        results.append({
            "point_index": idx,
            "location": points[idx],
            "total_population_2025": total_population_2025,
            "total_population_2029": total_population_2029,
            "total_population_2030": total_population_2030,
            "nearby_buildings": nearby_buildings
        })

    return results


# Example usage
results = estimate_population_for_lattice_optimized(
    buildings_with_population, lattice_points, standard_variables['standard_variables']['cutoff_radius']['value'])

In [13]:

# Display results
for result in results[:5]:
    print(f"Point {result['point_index']} (Location: {result['location']}):")
    print(f"  Total population (2025): {result['total_population_2025']}")
    print(f"  Total population (2029): {result['total_population_2029']}")
    print(f"  Total population (2030): {result['total_population_2030']}")

Point 0 (Location: (-22.083955, 64.033992)):
  Total population (2025): 0.0
  Total population (2029): 0.0
  Total population (2030): 0.0
Point 1 (Location: (-22.083056684715878, 64.033992)):
  Total population (2025): 0.0
  Total population (2029): 0.0
  Total population (2030): 0.0
Point 2 (Location: (-22.08215836943176, 64.033992)):
  Total population (2025): 0.0
  Total population (2029): 0.0
  Total population (2030): 0.0
Point 3 (Location: (-22.08126005414764, 64.033992)):
  Total population (2025): 0.0
  Total population (2029): 0.0
  Total population (2030): 0.0
Point 4 (Location: (-22.08036173886352, 64.033992)):
  Total population (2025): 0.0
  Total population (2029): 0.0
  Total population (2030): 0.0


In [14]:
# Prepare the results for saving
population_results_list = []
for result in results:
    population_results_list.append({
        "point_index": result["point_index"],
        "latitude": result["location"][1],
        "longitude": result["location"][0],
        "total_population_2025": result["total_population_2025"],
        "total_population_2029": result["total_population_2029"],
        "total_population_2030": result["total_population_2030"]
    })

# Convert to DataFrame
population_results_df = pd.DataFrame(population_results_list)

# Save as CSV
population_results_df.to_csv(
    "../data/lattice_population_proximity.csv", index=False)

print("Population results have been saved as '../data/lattice_population_proximity.csv'.")

Population results have been saved as '../data/lattice_population_proximity.csv'.


In [15]:
poi_data = gpd.read_file("../data/poi_data.geojson")
display(poi_data)

,Name,Latitude,Longitude,Weight,Reviews,Type,category,score,normalized_score,sigmoid_normalized_score,geometry
0,Whales of Iceland,64.155407,-21.948319,3.0,2593,Exhibition,Tourism,2593.0,0.109710,0.527400,POINT (-21.94832 64.15541)
1,Lavashow,64.156987,-21.942981,3.0,2309,Exhibition,Tourism,2309.0,0.097694,0.524404,POINT (-21.94298 64.15699)
2,Saga Museum,64.152589,-21.951381,3.0,1709,Museum,Tourism,1709.0,0.072308,0.518069,POINT (-21.95138 64.15259)
3,Reykjavík Maritime Museum,64.153298,-21.948718,3.0,940,Museum,Tourism,940.0,0.039772,0.509942,POINT (-21.94872 64.15330)
4,Árbær Open Air Museum,64.118911,-21.816251,3.0,1099,Museum,Tourism,1099.0,0.046499,0.511623,POINT (-21.81625 64.11891)
...,...,...,...,...,...,...,...,...,...,...,...
274,Reykjavík Terminal,64.134114,-21.920854,1.0,245,None,Trans. Hub,2450.0,0.103660,0.525892,POINT (-21.92085 64.13411)
275,Destination Blue Lagoon,64.133936,-21.920063,1.0,52,None,Trans. Hub,520.0,0.022001,0.505500,POINT (-21.92006 64.13394)
276,Airport Direct,64.134190,-21.920156,2.0,267,None,Trans. Hub,2670.0,0.112968,0.528212,POINT (-21.92016 64.13419)
277,Viðey Ferry Terminal,64.156234,-21.867257,1.0,24,None,Trans. Hub,240.0,0.010154,0.502539,POINT (-21.86726 64.15623)


In [16]:

def estimate_normalized_score_for_lattice(poi_data, points, radius):
    """
    Efficiently estimate the total normalized score for a lattice of points.

    Parameters:
    - poi_data (GeoDataFrame): GeoDataFrame containing POI data with a 'normalized_score' column.
    - points (list of tuples): List of (longitude, latitude) tuples representing locations to search near.
    - radius (float): The search radius in meters.

    Returns:
    - results (list of dict): List containing total normalized score and nearby POIs for each point.
    """
    # Reproject GeoDataFrame to metric CRS only once
    poi_metric = poi_data.to_crs(epsg=3857)
    spatial_index = poi_metric.sindex  # Build spatial index once

    # Reproject all points to metric CRS at once
    points_metric = gpd.GeoSeries(
        [Point(pt) for pt in points], crs=poi_data.crs
    ).to_crs(epsg=3857)

    results = []
    for idx, point in enumerate(points_metric):
        # Create a buffer around the point
        search_area = point.buffer(radius)

        # Use the spatial index to find potential matches
        possible_matches_index = list(
            spatial_index.intersection(search_area.bounds))
        possible_matches = poi_metric.iloc[possible_matches_index]

        # Filter with exact intersection
        nearby_pois = possible_matches[possible_matches.intersects(
            search_area)]

        # Calculate the total normalized score
        total_normalized_score = nearby_pois['normalized_score'].sum()

        results.append({
            "point_index": idx,
            "location": points[idx],
            "total_normalized_score": total_normalized_score,
            "nearby_pois": nearby_pois
        })

    return results

In [19]:
results = estimate_normalized_score_for_lattice(
    poi_data, lattice_points, standard_variables['standard_variables']['cutoff_radius']['value'])

# Prepare the results for saving
normalized_score_results_list = []
for result in results:
    normalized_score_results_list.append({
        "point_index": result["point_index"],
        "latitude": result["location"][1],
        "longitude": result["location"][0],
        "total_normalized_score": result["total_normalized_score"]
    })

# Convert to DataFrame
normalized_score_results_df = pd.DataFrame(normalized_score_results_list)

# Save as CSV
normalized_score_results_df.to_csv(
    "../data/lattice_normalized_score.csv", index=False)

print("Normalized score results have been saved as 'lattice_normalized_score.csv'.")

Normalized score results have been saved as 'lattice_normalized_score.csv'.


In [20]:
final_score_df = population_results_df[[
    'point_index', 'latitude', 'longitude']].copy()
final_score_df['final_score'] = 0.1*0.001*results_df['total_capacity']+0.001*population_results_df['total_population_2025']+0.001 * \
    population_results_df['total_population_2029']+0.001*population_results_df['total_population_2030'] + \
    100*normalized_score_results_df['total_normalized_score']
display(final_score_df)

final_score_df.to_csv("../data/lattice_final_score.csv", index=False)

,point_index,latitude,longitude,final_score
0,0,64.033992,-22.083955,0.000000
1,1,64.033992,-22.083057,0.000000
2,2,64.033992,-22.082158,0.000000
3,3,64.033992,-22.081260,0.000000
4,4,64.033992,-22.080362,0.000000
...,...,...,...,...
195619,195619,64.188920,-21.644679,0.000000
195620,195620,64.188920,-21.643781,0.001368
195621,195621,64.188920,-21.642882,0.001368
195622,195622,64.188920,-21.641984,0.001368
